# エンジニアのためのベイズ推定入門

## 確率論的プログラミングのすすめ

------------------

# 目次

## 第0部 準備

- はじめに
    - チュートリアルの進め方
    - 対象
- 参考文献，参考講義紹介
- セットアップ

## 第1部 ベイズ基礎

- 結合確率，周辺確率，条件付き確率
- ベイズの定理の導出
- ベイズ更新と事前確率・事後確率

## 第2部 モンテカルロ法と推定

- モンテカルロサンプリングとパラメータ推定
- ベイズの定理 + モンテカルロ -> 確率論的プログラミング
- 点推定，区間推定
    - EAP
    - MAP
    - MED
    - 区間推定
- マルコフ鎖モンテカルロ(MCMC)のさわり
	- メトロポリス・ヘイスティング法
	- ギブスサンプリング法
    - スライスサンプリング
	- ハミルトニアンモンテカルロ法
- 計算機実装
    - pymc3
    - pystan

## 第3部 モデリング入門
- 確率論的プログラマの仕事
- コイン投げ - pymc3
- コイン投げ - pystan
- モデリングの素振り練習
    - 正規分布 normal: pymc3, pystan
    - 線形回帰 linreg: pymc3, pystan
    - ロジステック回帰 gender: pymc3, pystan
    - 階層ベイズ gelmanradon: pymc3

-------------------------

# 第0部 準備

## はじめに

### チュートリアルの進め方

- 教材となるJupyter Notebookをダウンロード，またはDocker ImageをDocker Hubからpull，~~さらには今回の講習で利用するJupyterHubサーバー~~または以下のnbviewerにアクセスします．

http://nbviewer.ipython.org/github/bakfoo/pyconjp/tree/master/

- 教材，および捕捉資料はプロジェクタで投影します．
- Jupyter Notebookのコードを一つ一つ実行させながら，確率論的プログラミングを理解していきます．
- 質問は随時受け付けますが，時間内に答えることが難しい質問は後に回してまとめて回答します．

### 対象

- ベイズ推定の基本身につけたいと思っているエンジニア
- 確率論的プログラミングが気になっている全ての人
- Python / NumPy / SciPy を使ったプログラミングを何度か自分の力で書き上げた方，またはそれに必要な知識を持った方
- 大学初年度程度の数学（線形代数，確率論，統計学）の知識があると理解がスムーズになります．

## 参考文献，参考講義紹介

### 確率論的プログラミング入門/参考文献
- John Kruschke, Doing Bayesian Data Analysis 1st ed.

![alt text](fig/doing.jpg "doging")

- Gelman et al., Bayesian Data Analysis 3rd ed.

![alt text](fig/bda.jpg "bda")

- 豊田秀樹編著, 基礎からのベイズ統計学 - ハミルトニアンモンテカルロ法による実践的入門

![alt text](fig/toyoda.jpg "bda")

- C.P.ロバート & G. カセーラ, Rによるモンテカルロ法入門

![alt text](fig/robert.jpg "bda")

- Cliburn Chan & Janice McCarthy STA663 http://people.duke.edu/~ccc14/sta-663-2015/

### 確率論的プログラミング入門/MCMC HMC ビデオチュートリアル

修士の大学院生レベルですが，確率論的プログラミング入門として，以下のビデオチュートリアルは非常に良いです．

- Harvard Extentions AM207
    - Monte Carlo Methods, Stochastic Optimization (Verena Kaynig-Fittkau and Pavlos Protopapas) http://am207.org/
        - Python2, numpy/scipyのバニラな実装

- Machine Learning Summer School in Sydney, 2015 http://rp-www.cs.usyd.edu.au/~mlss/
    - Bayesian Inference and MCMC (Bob Carpenter) https://youtu.be/qQFF4tPgeWI
    - Stan Hands-on (Bob Carpenter) https://youtu.be/6NXRCtWQNMg
- Machine Learning Summer School in Iceland, 2014 http://mlss2014.hiit.fi/slides.php
    - Hamiltonian Monte Carlo and Stan (Michael Betancourt) https://www.youtube.com/watch?v=pHsuIaPbNbY  
    - Hamiltonian Monte Carlo and Stan 2 (Michael Betancourt) https://www.youtube.com/watch?v=xWQpEAyI5s8 https://www.youtube.com/watch?v=xWQpEAyI5s8

In [7]:
from IPython.display import YouTubeVideo

# Bob Carpenter: "Bayesian Inference and MCMC" in MLSS Sydney 2015
YouTubeVideo('qQFF4tPgeWI')

In [6]:
# Bob Carpenter: "Stan Hands-on" in MLSS Sydney 2015
YouTubeVideo('6NXRCtWQNMg')

In [8]:
# Michael Betancourt: "Hamiltonian Monte Carlo and Stan" in MLSS Iceland 2014
YouTubeVideo('pHsuIaPbNbY')

In [9]:
# Michael Betancourt: "Hamiltonian Monte Carlo and Stan" in MLSS Iceland 2014
YouTubeVideo('xWQpEAyI5s8')

## セットアップ

### 0 チュートリアル環境の構築前の注意

確率論的プログラミングはまだ若い分野ですので，計算環境の構築方法が成熟していません．チュートリアルではpymc3やpystanを利用しますが，それらの開発者は基本的にUbuntuにAnaconda Pythonを利用してるので，まともに動作する環境はどうしてもUbuntu + Anacondaが中心になってしまいます．以下に構築前の注意を列挙します．

- Windowsで確率論的プログラミングを行うことは，pymc3のtheanoのGPUの問題，pystanのプロセス制限の問題等，制約が多すぎて困難な道となります．Windowsを利用する場合，VMWare, VirtualBox, Vagrant等の仮想環境またはDocker等のコンテナ技術を利用してLinuxを用意するほうが精神衛生上望ましいと考えています．
- Python環境の構築はAnacondaディストリビューションを前提とします．素のPythonから確率論的プログラミングの環境を構築することは，数々の罠に嵌まる可能性が否定できませんので推奨できません．
- もしも，チュートリアル当日までに環境を構築できなくても問題ありません．チュートリアルはJupyter Notebookで行いますので，Docker Imageを用意~~するか，JupyterHub等によりWebブラウザからチュートリアルマテリアルにアクセス~~し実行できるようにします．
- 皆様には大変申し訳ございませんが，JupyterHubが技術的理由により用意できませんでした．お詫びを申しあげます．代わりに以下のnbviewerで代替させていただきます．

http://nbviewer.ipython.org/github/bakfoo/pyconjp/tree/master/

### 1 Linux/OSX環境に直接チュートリアル環境の構築する方法

それぞれのOS環境にて，C/C++のビルドができるところまで完了している，というのがチュートリアル環境を構築するための大前提です．たとえばUbuntuでは，

```
apt-get install build-essential
```

が完了している，ということです．OSX環境であればXcodeとコマンドラインツールがインストール済みで，コマンドラインでgccが動作するということです．

#### 1.1 Python環境の構築

以下のサイトからダウンロードしてAnaconda Pythonをインストールしてください．Python3.4を使います．

https://www.continuum.io/downloads

すでにAnaconda環境がある場合は，以下のようにpython環境を新しくすれば，既存の環境を壊すことなくチュートリアルの環境が構築できます．

```
conda create -n pycon python=3.4
source activate pycon
conda install anaconda
```

#### 1.2 pymc3のインストール

以下の手順でpipを利用してpymc3を構築します．ポイントはtheano, pymc3ともに最新版のリポジトリのものを利用しないとバグのために動作しない，ということです．インストール時にc/c++のビルドを行います．公式インストール手順：https://pymc-devs.github.io/pymc3/getting_started/

```
pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git
pip install --process-dependency-links git+https://github.com/pymc-devs/pymc3
pip install pasty
pip install seaborn
```

#### 1.3 pystanのインストール

次に以下の手順でpipを利用してpystanを構築します．インストール時にc/c++のビルドを行います．公式インストール手順：https://pystan.readthedocs.org/en/latest/getting_started.html

```
pip install pystan
```

以上でチュートリアル環境の構築は終了です．

### 2 Dockerを利用してチュートリアル環境を構築する方法

#### 2.1 VirtualBoxをダウンロードしてインストール

- https://www.virtualbox.org/wiki/Downloads

#### 2.2 Docker Toolboxをダウンロードしてインストール

- https://www.docker.com/toolbox

#### 2.3 以下のコマンドでDockerコンテナ起動

注意: 下記にある三番目のdocker runの"[LOCAL DIR]”の部分はご自分のディレクトリを指定ください．

```
docker-machine create --driver virtualbox --virtualbox-memory 3081 dev

eval "$(docker-machine env dev)"

docker run -t -i -v [LOCAL DIR]:/notebook -p 8888:8888 yutakashino/pyconjp:v1.2 /bin/bash

root@d3c2644d9fb3:/notebook# git clone https://github.com/bakfoo/pyconjp.git
root@d3c2644d9fb3:/notebook# ipython notebook --ip=* --port=8888
```

#### 2.4 ブラウザでアクセス

- ipは"docker-machine ip dev"により調べる
- portは8888
- 
```
http://192.168.99.101:8888/tree/pyconjp
```